In [1]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init_common as x
import simulate_corona_multi as sc
import simulate_corona_senior_multi as scs
import multiprocessing as mp
import save_part as sp

## setting
alpha_option_type =  3# alpha: 1=high 2=med 3=low 
x.R0_type = 0
out_filename_root,out_fig_root,out_stat_root=x.setup_paths(x.R0_type)
out_filename_pct0 = out_filename_root+"/"+x.out_filename_pct0
out_filename_raw0 = out_filename_root+"/"+x.out_filename_raw0

if x.R0_type==0:    
    import init_spec_gamma as xx
    import simulate_corona_multi as sf
elif x.R0_type==1:
    import init_spec_gamma as xx
    import simulate_corona_senior_multi as sf 

    
# print(out_filename_raw0)

## initialize
np.random.seed(123)
pct_data=[]
raw_data=[]

def run_simulation(alpha_vec,fun):
    ra = []
    rb = []
    cores_cnt = mp.cpu_count()
    pool = mp.Pool(cores_cnt)
    tic = time.perf_counter()
    res = pool.map(fun,alpha_vec)
    toc = time.perf_counter()
    pool.close()
    dt_avg = (toc-tic)/x.N_simul
    for k in np.arange(x.N_simul):
        ra0,rb0 = res[k]
        ra.append(ra0)
        rb.append(rb0)
    return ra,rb,dt_avg 

def save_results():
    filename_pct = out_filename_pct0+file_ext+".pickle"
    fid = open(filename_pct,'wb')
    pickle.dump(pct_data,fid)
    fid.close()
    filename_raw = out_filename_raw0+file_ext
    sp.save_file_pickle(raw_data,filename_raw)
    

opt_ok = True
if alpha_option_type == 1: # high alpha
    alpha_val = x.public_trans_high
    fnc_name = sf.simul
    file_ext = "_high"
elif alpha_option_type == 2: # medium alpha
    alpha_val = x.public_trans_mid
    fnc_name = sf.simul
    file_ext = "_med"
elif alpha_option_type == 3: # low alpha
    alpha_val = x.public_trans_low
    fnc_name = sf.simul
    file_ext = "_low"
elif option_type == 4: # seniors
    alpha_val = x.public_trans_low
    fnc_name = sf.simul
    file_ext = "_low_senior"
else:
    print("Incorrect option")
    opt_ok = False

print("Option type: "+str(alpha_option_type))
print("R0:"+str(x.R0))
print("alpha:"+str(alpha_val))
print("beta:"+str(xx.beta))
if opt_ok:
    alpha_vec = np.full(x.N_simul,alpha_val)

    ## run all simulations and 
    print("Running simulation ... ",end="",flush=True)
    pct_data,raw_data,tpi=run_simulation(alpha_vec,fnc_name)
    print("Finished")
    print("Time per iteration (sec.): "+str(tpi))

    ## save results
    print("Saving results ...",end="",flush=True)
    save_results()
    print("Finished")
    

Option type: 3
R0:2.4
alpha:0.6
beta:0.48
Running simulation ... Finished
Time per iteration (sec.): 5.350774074624999
Saving results ...Finished
